In [1]:
using LinearAlgebra
import Plots
import Roots # Root finding, I am using root_find
import Random
import StatsBase

In [2]:
# The System is defined by the jump operators and the Hamiltonian
struct System
    NLEVELS::Int64 #Number of levels of the system
    H::Matrix{ComplexF64} # Hamiltonian
    Ls::Vector{Matrix{ComplexF64}} # List of jump operators
    J::Matrix{ComplexF64} # Sum of Jump operators
    Heff::Matrix{ComplexF64} # Effective Hamiltonian
    # ExpHeff::Function # Function that calculates the exponential of Heff at tau
    # JumpSuperOperators::Vector{Function}  # Jump super operators
    # NoJumpSuperOperator::Function # No-Jump superoperator
    # Inner Constructor
    function System(H::Matrix{ComplexF64}, Ls::Vector{Matrix{ComplexF64}})
        NLEVELS = size(H)[1] 
        NCHANNELS = size(Ls)[1] # Number of jump channels
        J = zeros(ComplexF64, NLEVELS, NLEVELS) 
        for L in Ls
            J = J + adjoint(L)*L
        end 
        CurvyLs = Vector{Function}(undef, NCHANNELS)
       # for k in 1:NCHANNELS
       #     CurvyLs[k] = rho::Matrix{ComplexF64} -> Ls[k]*rho*adjoint(Ls[k])
       # end 
        He = H - 0.5im*J
       # expHe(tau::Float64) =  expm(-1im*tau*He)
       # function expcurvyL0(rho::Matrix{ComplexF64}, tau::Float64)
       #     A = expHe(tau)
       #     return A*rho*adjoint(A)
       # end 
       # new(NLEVELS, H, Ls, J, He, expHe, CurvyLs, expcurvyL0)
        new(NLEVELS, H, Ls, J, He)
    end 
end 

In [3]:
# Trajectory Struct
struct Trajectory
    times::Vector{Float64} # detection times
    states::Vector{Vector{ComplexF64}} #States after jump
    labels::Vector{Int64} # Labels 
end 

In [4]:
# Simulation Struct, it contains the data necessary to run the simulation
# and the precomputed values
struct SimulParameters
    psi0::Vector{Float64}
    nsamples::Int64 # Number of samples in the finegrid
    seed::Int64 # seed
    ntraj::Int64 # Number of trajectories
    multiplier::Float64 # Multiplier to use in the fine grid
    tf::Float64 # Final time
    dt::Float64 # time step for the finegrid
    eps::Float64 # Tolerance for passing WTD normalziation
    function SimulParameters(psi0::Vector{ComplexF64}, tf::Float64,
        s::Int64, ntraj::Int64, nsamples::Int64=10000, m::Int64=10 ,eps::Float64=1e-3)
        deltat = m*tf/nsamples
        new(psi0, nsamples, s, ntraj, m, tf, deltat, eps)
    end 
end 

In [5]:
############# Precomputation routine ######################
# Input: 
# 1. From the System: J and Heff
# 2. From the simulation parameters: tf, nsamples and the multiplier
# Output: (ts, Qs), a tuple with the finegrid and the precomputed values of the transformed J
function precompute(J::Matrix{ComplexF64}, Heff::Matrix{ComplexF64}, 
        tf::Float64, nsamples::Int64, multiplier::Int64)
        ts = LinRange(0, multiplier*tf, nsamples)
        Qs = Vector{Matrix{ComplexF64}}(undef, nsamples) 
        for k in 1:NSAMPLES
            expm = exp(-1im*ts[k]*Heff) 
            Qs[k] = expm * J * adjoint(expm)
        end 
        return ts, Qs
end 
############# Single Trajectory Routine ######################
# Input:
# 1. From the System: Ls and Heff
# 2. From the Simulation Parameters: psi0, tf, nsamples, dt and eps
# Output:
# A trajectory object with the data of the trajectory
function run_single_trajectory(sys::System, params::SimulParameters)
    # System 
    J = sys.J
    Heff = sys.Heff
    Ls = sys.Ls
    # Simulation parameters
    tf = params.tf 
    nsamples = params.nsamples
    dt = params.dt
    eps = params.eps
    # Store 
    W = zeros(Float64, nsamples) # Vector to store the weights of the fine grid
    times = Vector{Vector{ComplexF64}}()
    labels = Vector{Int64}()
    states = Vector{Vector{ComplexF64}}()
 
   # 1. Set Initial condition
    psi = params.psi0
    t = 0
    # 2. Precomputing 
    ts, Qs =  precompute(J, Heff, tf, nsamples, params.multiplier)
    # 3. Run the trajectory
    while t < tf
    # 1. Calculate the WTD for the state, these act as weights
        for k in 1:nsamples
           W[k] = real(dot(conj.(psi), Qs[k]*psi)) 
        end 
        # 1.a We must verify if we got a dark state, that can be cheked by 
        # looking at the normalization of the QTD
        if abs(sum(W)*dt - 1) > eps
            break
        end 
        # 2. Sample jump time
        tau = StatsBase.sample(ts, weights(W))
        t = tau + t
        if t>tf # If the next jump happens after tf, stop 
            break
        end 
        psi_tilde = expH_eff(tau) * L1*psi # State without normalization
        psi = psi_tilde / norm(psi_tilde)
        push!(states, psi)
        push!(labels, 1)
        push!(times, t)
    end 
    return Trajectory(times, states, labels) 
end 

run_single_trajectory (generic function with 1 method)

In [6]:
################### Qubit decay example ######################
#### 1. Create the system instance
gamma = 1
sigma_m = [[0.0+0im, 0]  [1, 0]]
sys = System( zeros(ComplexF64, 2, 2), # Hamiltonian
    [sqrt(gamma)*sigma_m ]) #Jump Operators
#### 2. Create the simulation parameters instance
psi0 = zeros(ComplexF64, 2)
psi0[2] = 1 # Initial condition
tf = 1.5
seed = 1
ntraj = 50
# We are using the default conditions:
# nsamples=10000  ntraj=100 seed=1 eps=1e-3 multiplier=10.0
simulparams = SimulParameters(psi0, tf, seed, ntraj)
### 3. Run a single trajectory
run_single_trajectory(sys, simulparams)

LoadError: MethodError: no method matching precompute(::Matrix{ComplexF64}, ::Matrix{ComplexF64}, ::Float64, ::Int64, ::Float64)
The function `precompute` exists, but no method is defined for this combination of argument types.

[0mClosest candidates are:
[0m  precompute(::Matrix{ComplexF64}, ::Matrix{ComplexF64}, ::Float64, ::Int64, [91m::Int64[39m)
[0m[90m   @[39m [35mMain[39m [90m[4mIn[5]:6[24m[39m


In [421]:
################### Setup ###############################
#1. Simulation conditions
NSAMPLES = 10000 # Number of points in the fine-grid, for the precomputation
seed = 1 # Seed, clearly
W = zeros(NSAMPLES) # To store the weights in sampling
#const Trajectory = Vector{Tuple{Float64, Vector{Float64}}} # Alias for trajectory 
observed_trajectories = Vector{Trajectory}() # Vector with the trajectories
EPS = 1e-3
NTRAJ = 50 # Number of trajectories
tf = 1 # Final time
# Paramters
gamma = 1
# Initial State
psi0 = zeros(2)
psi0[2] = 1 # Initial condition
NLEVELS = size(psi0)[1]
# Jump Operators and related
sigma_m = [[0.0+0im, 0]  [1, 0]]
L1 = sqrt(gamma) * sigma_m
J = adjoint(L1)*L1
H_eff = -0.5im * J
expH_eff(tau) = exp(-1im*H_eff*tau) # Exponential of -i*tau*H_eff

################### Precomputation ######################
# Following Radaelli's advice, I'll choose a fine grid of time 
# To precompute J
multiplier = 10 # Factor to multiply by tf and create the sample
ts = LinRange(0, multiplier*tf, NSAMPLES)
Qs = Vector{Matrix{ComplexF64}}(undef, NSAMPLES)
dt = multiplier*tf/NSAMPLES
for k in 1:NSAMPLES
    expm = expH_eff(ts[k]) 
    Qs[k] = expm * J * adjoint(expm)
end 

@time begin
for j in 1:NTRAJ
Random.seed!(seed+j)
############### Running the Trajectory #################
# 0. Set Initial condition
    trajectory = Vector{Tuple{Float64,Vector{ComplexF64}}}()
    psi = zeros(2)
    psi = psi0
    t = 0
    while t < tf
    # 1. Calculate the WTD for the state, these act as weights
        for k in 1:NSAMPLES
           W[k] = real(dot(conj.(psi), Qs[k]*psi)) 
        end 
        # 1.a We must verify if we got a dark state, that can be cheked by 
        # looking at the normalization of the QTD
        if abs(sum(W)*dt - 1) > EPS
            break
        end 
        # 2. Sample jump time
        tau = StatsBase.sample(ts, weights(W))
        t = tau + t
        if t>tf # If the next jump happens after tf, stop 
            break
        end 
        psi_tilde = expH_eff(tau) * L1*psi # State without normalization
        psi = psi_tilde / norm(psi_tilde)
        push!(trajectory, (t, psi))
    end 
    push!(observed_trajectories, trajectory)
end 
end 

  0.666617 seconds (9.61 M allocations: 302.479 MiB, 11.33% gc time)


In [ ]:
function SampleSingleTrajectory
end